In [2]:
!pip install langchain_community
!pip install langchain

In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 6.3 MB/s eta 0:00:00


In [4]:
!pip install openai

In [6]:
!pip install chromadb

In [7]:
!pip install tiktoken

In [8]:
import openai
import chromadb

In [16]:
!pip show langchain
!pip install langchain-text-splitters

Name: langchain
Version: 1.1.2
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


In [19]:
# import libraries
import os
from langchain_community.document_loaders import PyPDFLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain-text-splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

SyntaxError: invalid syntax (ipython-input-4068816712.py, line 5)

In [ ]:
OPENAI_API_KEY = "" # add your OpenAI API Key
# for this example I used Alphabet Inc 10-K Report 2022
# https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm
DOC_PATH = "/content/sample_data/alphabet_10K_2022.pdf"
CHROMA_PATH = "Chroma"

In [ ]:
# ----- Data Indexing Process -----
# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# get OpenAI Embedding model
embeddings = OpenAIEmbeddings(openai_api_key="")

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

In [ ]:
# ----- Retrieval and Generation Process -----

# this is an example of a user question (query)
query = 'what are the top risks mentioned in the document?'

# retrieve context - top 5 most relevant (closests) chunks to the query vector
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

In [ ]:
# you can use a prompt template
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

# call LLM model to generate the answer based on the given context and query
model = ChatOpenAI(openai_api_key="")
response_text = model.predict(prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
response_text

'The top risks mentioned in the document are:\n1. Interference, interruptions, and failures caused by various factors like modifications, upgrades, attacks, disasters, and climate change.\n2. Decline in the value of investments leading to harm in financial condition and operating results.\n3. Harm to business due to lack of adoption of products and services on evolving platforms and devices.\n4. Government and legal exposure, reputation harm, and impairment in attracting and retaining users or customers.\n5. Security issues like errors, vulnerabilities, interruptions, or failures in services or systems.\n6. Risks associated with international operations.\n7. Geopolitical tensions or armed conflicts increasing risks.\n8. Security issues related to employee or insider error, system errors, or vulnerabilities.'